In [1]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline


from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [2]:
#DATA CLEANING 

In [3]:
df_chartered_bank_i = pd.read_csv('../Resources/Datasets/chartered_bank_interest.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df_chartered_bank_i.head()
# Convert daily data to monthly through the resample function
df_chartered_bank_i.index = pd.to_datetime(df_chartered_bank_i.index)
df_chartered_bank_i_cleaned = df_chartered_bank_i.resample('1Q').mean()
df_chartered_bank_i_cleaned.tail()
df_chartered_bank_i_cleaned.head()
df_chartered_bank_i_cleaned.drop(df_chartered_bank_i_cleaned.columns.difference(['5-year Conventional mortgage', 'Prime rate']), 1, inplace=True)
df_chartered_bank_i_cleaned.head()
df_Residential_mortgages = pd.read_csv('../Resources/Datasets/Credit liabilities of households.csv')
df_Residential_mortgages.head
res_mortgage_transp = df_Residential_mortgages.T
res_mortgage_transp.tail()

# Remove unwanted columns
res_mortgage_transp = res_mortgage_transp.truncate(before='35', after='35', axis=1)
res_mortgage_transp.tail()

# Reset index and change headers
res_mortgage_transp = res_mortgage_transp.reset_index(drop=False)
new_header = res_mortgage_transp.iloc[0] #grab the first row for the header
res_mortgage_transp = res_mortgage_transp[1:] #take the data less the header row
res_mortgage_transp.columns = new_header #set the header row as the dataframe header
res_mortgage_transp.tail()

# Rename the columns
res_mortgage_transp = res_mortgage_transp.rename(columns={'Residential mortgages 14': 'Residential Mortgages'})

# Review data types
res_mortgage_transp.dtypes

# Convert data types to required format
res_mortgage_transp['Date'] = res_mortgage_transp['Date'].astype('datetime64[ns]')
res_mortgage_transp['Residential Mortgages'] = res_mortgage_transp['Residential Mortgages'].astype(float)
res_mortgage_transp.info()

# Convert monthly data to quarterly 
res_mortgage_transp.Date = pd.to_datetime(res_mortgage_transp.Date)

#setting the index to the Date column 
res_mortgage_transp.set_index('Date', inplace=True)
res_mortgage_transp.tail()

#using resample to take Quarterly data 
res_mortgage_cleaned = res_mortgage_transp.resample('1Q').mean()
res_mortgage_cleaned['Residential Mortgages']=res_mortgage_cleaned['Residential Mortgages'].apply(lambda x:round(x,2))
res_mortgage_cleaned.tail()

# House afforability index 
df_House_afford_index  = pd.read_csv('../Resources/Datasets/AffordabilityIndex date.csv')
df_House_afford_index.head()

#here we will be doing some cleaning from this dataset and taking only the Housing Affordability Index
df_House_afford_index = df_House_afford_index.drop(columns=['New housing price index'])
df_House_afford_index.tail()

df_House_afford_index['Date'] = df_House_afford_index['Date'].astype('datetime64[ns]')
df_House_afford_index.dtypes

df_House_afford_index.set_index('Date', inplace=True)
df_House_afford_index_cleaned = df_House_afford_index.resample('1Q').last()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 1 to 381
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   381 non-null    datetime64[ns]
 1   Residential Mortgages  381 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 6.1 KB


In [4]:
#check null values 
res_mortgage_cleaned.isnull().sum()
#drop null values 

#check again for null values 
df_House_afford_index_cleaned.isnull().sum()

#removing null values 
df_House_afford_index_cleaned = df_House_afford_index_cleaned.dropna().copy()

#check again for null values 
df_House_afford_index_cleaned.isnull().sum()

#check again for null values 
df_chartered_bank_i_cleaned.isnull().sum()

#removing null values 
df_chartered_bank_i_cleaned = df_chartered_bank_i_cleaned.dropna().copy()

#check again for null values 
df_chartered_bank_i_cleaned.isnull().sum()

Prime rate                      0
5-year Conventional mortgage    0
dtype: int64

In [5]:
df_chartered_bank_i_cleaned = df_chartered_bank_i_cleaned['1990-03-31' :'2021-06-30']
print(df_chartered_bank_i_cleaned.head())
print(df_chartered_bank_i_cleaned.tail())

# for Housing Affordability Index
#slice only the dates from 1990-03-31 TO 2021-06-30

df_House_afford_index_cleaned = df_House_afford_index_cleaned['1990-03-31' :'2021-06-30']
print(df_House_afford_index_cleaned.head())
print(df_House_afford_index_cleaned.tail())

# for Residential Mortgages
#we slice only the dates from 1990-03-31 TO 2021-06-30

res_mortgage_cleaned = res_mortgage_cleaned['1990-03-31' :'2021-06-30']
print(res_mortgage_cleaned.head())
print(res_mortgage_cleaned.tail())


#combine into a single dataframe
 
housing_analysis_combined_df = pd.concat([res_mortgage_cleaned, df_House_afford_index_cleaned, df_chartered_bank_i_cleaned], axis="columns", join="inner")

# Sort datetime index in ascending order (past to present)
housing_analysis_combined_df.sort_index(inplace=True)

# Display a few rows
housing_analysis_combined_df.tail()

            Prime rate  5-year Conventional mortgage
date                                                
1990-03-31   13.846154                     12.423077
1990-06-30   14.634615                     14.096154
1990-09-30   14.480769                     13.769231
1990-12-31   13.519231                     12.980769
1991-03-31   11.826923                     11.865385
            Prime rate  5-year Conventional mortgage
date                                                
2020-06-30        2.45                      4.993846
2020-09-30        2.45                      4.854286
2020-12-31        2.45                      4.790000
2021-03-31        2.45                      4.790000
2021-06-30        2.45                      4.790000
            Housing Affordability Index
Date                                   
1990-03-31                        0.479
1990-06-30                        0.527
1990-09-30                        0.494
1990-12-31                        0.467
1991-03-31        

,Residential Mortgages,Housing Affordability Index,Prime rate,5-year Conventional mortgage
2020-06-30,1575288.33,0.300,2.45,4.993846
2020-09-30,1604927.33,0.315,2.45,4.854286
2020-12-31,1639785.00,0.337,2.45,4.790000
2021-03-31,1661491.67,0.347,2.45,4.790000
2021-06-30,1708387.33,0.363,2.45,4.790000
